In [34]:
import os
import sys
import matplotlib.pyplot as plt
sys.path.append('../')
from utils.io import load_json, load
from utils.Logger import Logger
from utils.experiment_evaluator import Evaluator
import plotly
import plotly.graph_objs as go

plotly.offline.init_notebook_mode(connected=True)

data_directory = '../experiments_results/'
experiments = [dI for dI in os.listdir(data_directory) if os.path.isdir(os.path.join(data_directory, dI))]
# experiments = [dI for dI in experiments if not dI[0] == '_']
experiments.sort(key=lambda x: x.lower())

In [52]:
import pandas as pd
import seaborn as sns
sns.set(style="darkgrid") 

In [59]:
def get_styles(exp):
    exp = exp.lower()
    if 'ar_lstm' in exp:
        style = '-'
    elif 'convls' in exp:
        style = ':'
    elif 'resnet' in exp:
        style =  '--'
    return style

In [77]:
evaluators = {}
for experiment in experiments:
#     print(experiment)
    try:
        evaluators[experiment] = load('../experiments_results/' + experiment + '/pickles/evaluator_Illumination_135_sp_15.pickle')
    except:
        print(experiment)
        pass

In [78]:
# fig = plt.figure(figsize=(20,20)).add_axes()

# for exp, ev in evaluators.items():
# #     if not '_bad' in exp.lower():
# #         continue
#     if not 'ar_lst' in exp.lower():
#         continue
#     print(exp)
#     df = ev.get_rmse_values()
# #     sns.lineplot(x=df.index, y="mean", hue="Network",
# #              data=df, ax=fig, ci='sd')
#     x = df.index.values
#     y = df['mean'].values
#     style = get_styles(exp)
#     plt.plot(x, y, label=' '+exp, linewidth=2, linestyle=style)
# plt.legend()
# plt.show()

In [79]:
# fig = plt.figure(figsize=(20,20)).add_axes()

# for exp, ev in evaluators.items():
#     df = ev.get_ssim_values()
    
# #     sns.lineplot(x=df.index, y="mean", hue="Network",
# #              data=df, ax=fig, ci='sd')
#     x = df.index.values
#     y = df['mean'].values
#     style = get_styles(exp)
#     plt.plot(x, y, label=exp, linewidth=2, linestyle=style)
# plt.legend()
# plt.show()

In [80]:
traces = []
for exp, ev in evaluators.items():
#     if not '_bad' in exp.lower():
#         continue
#     if 'resnet' in exp.lower():
#         continue
    df = ev.get_rmse_values()

    x = df.index.values
    y = df['mean'].values
    traces.append(go.Scatter(x=x, y=y, name=exp,
                            visible = 'legendonly'))

layout = dict(
    legend=dict(
        y=0.9,
#         traceorder='reversed',
        font=dict(
            size=8
        )
    )
)

plotly.offline.iplot({
        "data": traces,
        "layout": layout
    })

In [50]:
traces = []
for exp, ev in evaluators.items():
#     if not '_bad' in exp.lower():
#         continue
#     if 'resnet' in exp.lower():
#         continue
    df = ev.get_ssim_values()

    x = df.index.values
    y = df['mean'].values
    error = None # df['std'].values
    traces.append(go.Scatter(x=x, y=y, name=exp, error_y=dict(array=error)))

layout = dict(
    legend=dict(
        y=0.9,
#         traceorder='reversed',
        font=dict(
            size=8
        )
    )
)

plotly.offline.iplot({
        "data": traces,
        "layout": layout
    })